In [1]:
from rich import print
import logging

logging.basicConfig(level=logging.INFO)

# Modelling with PyCaret

In the springtime project we use[PyCaret](https://github.com/pycaret/pycaret) to
train, evaluate and compare (machine learning) models. PyCaret is a Python
wrapper around several machine learning libraries and frameworks such as
[scikit-learn](https://scikit-learn.org/stable/) and
[XGBoost](https://xgboost.readthedocs.io/en/latest/).

For a (ML) model to work with PyCaret, it [needs to adhere
to](https://pycaret.gitbook.io/docs/learn-pycaret/faqs#can-i-add-my-own-custom-models-in-pycaret)
the [scikit-learn API](https://scikit-learn.org/stable/developers/develop.html).
This API specifies, for example, that each model must have a `fit` and a
`predict` method. It also specifies the expected structure of the input data.
This is why we need to make a big effort to standardize our datasets.

## Basic experiment

Let's see what a 'simple' experiment looks like. We'll load the same example
data as before, and compare a few 'standard' models. For now, we'll stick to all
the default settings of pycaret.


In [2]:
from springtime.datasets import PEP725Phenor, EOBS
from springtime.utils import germany, PointsFromOther, join_dataframes

years = [2000, 2002]

pep725 = PEP725Phenor(
    species="Syringa vulgaris",
    years=years,
    area=germany,
)

eobs = EOBS(
    area=germany,
    years=years,
    variables=["mean_temperature"],
    resample={"frequency": "M", "operator": "mean"},
    points=PointsFromOther(source="pep725"),
)

df_pep725 = pep725.load()
eobs.points.get_points(df_pep725)
df_eobs = eobs.load()
df = join_dataframes([df_pep725, df_eobs])
df.head()

INFO:springtime.datasets.eobs:Locating data
INFO:springtime.datasets.eobs:Looking for variable mean_temperature in period 2000-2002...
INFO:springtime.datasets.eobs:Found /home/peter/.cache/springtime/e-obs/tg_ens_mean_0.1deg_reg_1995-2010_v26.0e.nc


day  mean_temperature|31  mean_temperature|59  \
year geometry                                                                   
2000 POINT (10.00000 49.48330)  129             0.323548                  NaN   
     POINT (10.00000 50.85000)  120             0.943226                  NaN   
     POINT (10.00000 51.71670)  116             1.694194                  NaN   
     POINT (10.00000 52.10000)  120             2.531935                  NaN   
     POINT (10.00000 53.08330)  121             2.119677                  NaN   

                                mean_temperature|60  mean_temperature|90  \
year geometry                                                              
2000 POINT (10.00000 49.48330)             3.664483                  NaN   
     POINT (10.00000 50.85000)             3.795517                  NaN   
     POINT (10.00000 51.71670)             4.053448                  NaN   
     POINT (10.00000 52.10000)             4.937242                  NaN   
     POINT (10.00000 53.08330)             3.988276                  NaN   

                                mean_temperature|91  mean_temperature|120  \
year geometry                                                               
2000 POINT (10.00000 49.48330)             5.358709                   NaN   
     POINT (10.00000 50.85000)             5.660645                   NaN   
     POINT (10.00000 51.71670)             5.399354                   NaN   
     POINT (10.00000 52.10000)             5.771289                   NaN   
     POINT (10.00000 53.08330)             4.812258                   NaN   

                                mean_temperature|121  mean_temperature|151  \
year geometry                                                                
2000 POINT (10.00000 49.48330)              9.966999                   NaN   
     POINT (10.00000 50.85000)             10.001336                   NaN   
     POINT (10.00000 51.71670)             10.563000                   NaN   
     POINT (10.00000 52.10000)             10.993333                   NaN   
     POINT (10.00000 53.08330)              9.906999                   NaN   

                                mean_temperature|152  ...  \
year geometry                                         ...   
2000 POINT (10.00000 49.48330)             14.801293  ...   
     POINT (10.00000 50.85000)             14.421612  ...   
     POINT (10.00000 51.71670)             14.321937  ...   
     POINT (10.00000 52.10000)             14.817741  ...   
     POINT (10.00000 53.08330)             14.663547  ...   

                                mean_temperature|243  mean_temperature|244  \
year geometry                                                                
2000 POINT (10.00000 49.48330)                   NaN             18.310001   
     POINT (10.00000 50.85000)                   NaN             17.148067   
     POINT (10.00000 51.71670)                   NaN             17.539680   
     POINT (10.00000 52.10000)                   NaN             17.556454   
     POINT (10.00000 53.08330)                   NaN             16.573227   

                                mean_temperature|273  mean_temperature|274  \
year geometry                                                                
2000 POINT (10.00000 49.48330)                   NaN             13.865000   
     POINT (10.00000 50.85000)                   NaN             13.630667   
     POINT (10.00000 51.71670)                   NaN             14.346666   
     POINT (10.00000 52.10000)                   NaN             14.623999   
     POINT (10.00000 53.08330)                   NaN             13.515334   

                                mean_temperature|304  mean_temperature|305  \
year geometry                                                                
2000 POINT (10.00000 49.48330)                   NaN             10.093225   
     POINT (10.00000 50.85000)                   NaN              9.831611   
     POINT

In [3]:
from pycaret.regression import RegressionExperiment

df.reset_index(drop=True, inplace=True)  # drop index to avoid sorting errors

exp = RegressionExperiment()
exp.setup(data=df, target="day")
exp.compare_models(["lr", "rf", "dummy"], n_select=3)

,Description,Value
0,Session id,7530
1,Target,day
2,Target type,Regression
3,Original data shape,"(4729, 24)"
4,Transformed data shape,"(4729, 24)"
5,Transformed train set shape,"(3310, 24)"
6,Transformed test set shape,"(1419, 24)"
7,Numeric features,23
8,Rows with missing values,100.0%
9,Preprocess,True


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,4.2245,39.7006,6.2849,0.4740,0.0513,0.0344,0.8160
lr,Linear Regression,5.2901,48.3598,6.9462,0.3565,0.0564,0.0430,0.4250
dummy,Dummy Regressor,6.7257,75.9940,8.7000,-0.0041,0.0705,0.0548,0.0390


[RandomForestRegressor(n_jobs=-1, random_state=7530),
 LinearRegression(n_jobs=-1),
 DummyRegressor()]

Pycaret successfully executed the training of three models, and it concluded
that the random forest regressor performed best, with a mean absolute error of
about 4 days and an RMSE of 6.5 days. This seems okay, but the low correlation
coefficient (R2) suggests that the skill is still limited. Obviously, a more
in-depth investigation is necessary to conclude anything from this, but the
initial scoring is already a great start.

Notice the default settings of pycaret: it uses a 10-fold cross-validation
strategy and can do simple preprocessing tasks such as simple imputation of
missing data.


In [4]:
# TODO: eobs has NANs due to DOY matching of monthly resampled data with leap years.

## Custom estimators: interpretML

Any model that adheres to the scikit-learn API can be used in pycaret. For
example, we are interested in using [interpretML](https://interpret.ml/docs/).

Note: you need to have installed interpretML in your springtime environment for this to work:

```bash
pip install interpet
```


In [5]:
from interpret.glassbox import ExplainableBoostingRegressor

ebm = ExplainableBoostingRegressor()

exp = RegressionExperiment()
exp.setup(data=df, target="day")
exp.compare_models(["lr", "rf", "dummy", ebm], n_select=3)

,Description,Value
0,Session id,7202
1,Target,day
2,Target type,Regression
3,Original data shape,"(4729, 24)"
4,Transformed data shape,"(4729, 24)"
5,Transformed train set shape,"(3310, 24)"
6,Transformed test set shape,"(1419, 24)"
7,Numeric features,23
8,Rows with missing values,100.0%
9,Preprocess,True


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
3,ExplainableBoostingRegressor,3.9667,40.5677,6.3604,0.4824,0.0553,0.0331,1.0480
1,Random Forest Regressor,4.2388,44.1128,6.6274,0.4379,0.0573,0.0353,0.8130
0,Linear Regression,5.2808,51.7150,7.1857,0.3390,0.0613,0.0437,0.0450
2,Dummy Regressor,6.6979,78.6172,8.8596,-0.0040,0.0743,0.0553,0.0500


INFO:interpret.utils._native:EBM lib loading.
INFO:interpret.utils._native:Loading native on Linux | debug = False
INFO:interpret.utils._compressed_dataset:Creating native dataset
INFO:interpret.glassbox._ebm._bin:eval_terms
INFO:interpret.glassbox._ebm._bin:eval_terms
INFO:interpret.glassbox._ebm._bin:eval_terms
INFO:interpret.glassbox._ebm._bin:eval_terms
INFO:interpret.glassbox._ebm._bin:eval_terms
INFO:interpret.glassbox._ebm._bin:eval_terms
INFO:interpret.glassbox._ebm._bin:eval_terms
INFO:interpret.glassbox._ebm._bin:eval_terms
INFO:interpret.utils._compressed_dataset:Creating native dataset
INFO:interpret.glassbox._ebm._ebm:Estimating with FAST
INFO:interpret.glassbox._ebm._bin:eval_terms


[ExplainableBoostingRegressor(),
 RandomForestRegressor(n_jobs=-1, random_state=7202),
 LinearRegression(n_jobs=-1)]

Nice! Our explainable boosting machine outperformed the random forest regression. This is promising!


## Custom estimators: Mixed-effects models

Another interesting approach to (interpretable) modelling is the use of
mixed-effects models. We found an existing package,
[MERF](https://github.com/manifoldai/merf), that combines a random forest with
random effects. However, the package was not scikit-learn compatible out of the
box. Therefore, we've made an adaptated version called [DumME] which is fully
scikit-learn compatible and replaces the default model with the scikit-learn
dummy model. You can install this package with

```bash
pip install dumme
```

The nice thing is that you are not tied to the Dummy or Random Forest models for
the fixed effects. You can combine it with any other sklearn-compatible
(regression) model. Here, we create three mixed effects models and see how they
perform.


In [6]:
import numpy as np
from dumme.dumme import MixedEffectsModel
from pycaret.regression import RegressionExperiment
from sklearn.ensemble import RandomForestRegressor

# We need to add a cluster column, otherwise each unique sample will be treated
# as a cluster and the algorithm will be very slow.
df = df.copy()
df["cluster"] = np.random.randint(0, 3, len(df))

ebm = ExplainableBoostingRegressor()
me_dummy = MixedEffectsModel(max_iterations=2)
me_rf = MixedEffectsModel(RandomForestRegressor(), max_iterations=2)
me_ebm = MixedEffectsModel(ExplainableBoostingRegressor(), max_iterations=2)

exp = RegressionExperiment()
exp.setup(data=df, target="day")

# Notice: We can pass in the fit arguments for DumME, but that breaks the other models.
# exp.compare_models(["lr", "rf", "dummy", ebm, me_dummy, me_rf, me_ebm], n_select=3, fit_kwargs={"cluster_column": "cluster"})

# Therefore, instead, we rely on the default of DumME to use the last column as cluster column
exp.compare_models(["lr", "rf", "dummy", ebm, me_dummy, me_rf, me_ebm], n_select=3)

,Description,Value
0,Session id,3711
1,Target,day
2,Target type,Regression
3,Original data shape,"(4729, 25)"
4,Transformed data shape,"(4729, 25)"
5,Transformed train set shape,"(3310, 25)"
6,Transformed test set shape,"(1419, 25)"
7,Numeric features,24
8,Rows with missing values,100.0%
9,Preprocess,True


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
3,ExplainableBoostingRegressor,3.9866,40.5575,6.3355,0.5093,0.0548,0.0332,1.1060
6,MixedEffectsModel,3.9925,40.6097,6.3389,0.5088,0.0548,0.0333,7.0540
1,Random Forest Regressor,4.2263,43.3897,6.5584,0.4742,0.0564,0.0351,0.7830
5,MixedEffectsModel,4.2393,43.5772,6.5733,0.4718,0.0566,0.0352,6.4690
0,Linear Regression,5.3509,53.1597,7.2676,0.3515,0.0618,0.0442,0.0440
2,Dummy Regressor,6.8229,82.6911,9.0721,-0.0067,0.0760,0.0564,0.0390
4,MixedEffectsModel,6.8274,82.7419,9.0751,-0.0074,0.0760,0.0564,5.2550


INFO:interpret.utils._compressed_dataset:Creating native dataset
INFO:interpret.glassbox._ebm._bin:eval_terms
INFO:interpret.glassbox._ebm._bin:eval_terms
INFO:interpret.glassbox._ebm._bin:eval_terms
INFO:interpret.glassbox._ebm._bin:eval_terms
INFO:interpret.glassbox._ebm._bin:eval_terms
INFO:interpret.glassbox._ebm._bin:eval_terms
INFO:interpret.glassbox._ebm._bin:eval_terms
INFO:interpret.glassbox._ebm._bin:eval_terms
INFO:interpret.utils._compressed_dataset:Creating native dataset
INFO:interpret.glassbox._ebm._ebm:Estimating with FAST
INFO:interpret.glassbox._ebm._bin:eval_terms
INFO:interpret.utils._compressed_dataset:Creating native dataset
INFO:interpret.glassbox._ebm._bin:eval_terms
INFO:interpret.glassbox._ebm._bin:eval_terms
INFO:interpret.glassbox._ebm._bin:eval_terms
INFO:interpret.glassbox._ebm._bin:eval_terms
INFO:interpret.glassbox._ebm._bin:eval_terms
INFO:interpret.glassbox._ebm._bin:eval_terms
INFO:interpret.glassbox._ebm._bin:eval_terms
INFO:interpret.glassbox._ebm._

[ExplainableBoostingRegressor(),
 MixedEffectsModel(fe_model=ExplainableBoostingRegressor(), max_iterations=2),
 RandomForestRegressor(n_jobs=-1, random_state=3711)]

That's as far as the introductions go.

## Next steps: adding your own (physical) models

While many ML packages already adhere to the scikit-learn API, there are other
methods that do not. Particularly, we want to compare our ML-based approach with
more traditional physics-based models. In the next chapter, we will show how to
add a growing-degree day model to our framework.
